In [ ]:
# Read only the specified text column
df_discharge_text = pd.read_csv(path, usecols=[text_access], compression='gzip')

In [ ]:
# Keep only 5K records for computational efficiency.
df_discharge_text = df_discharge_text.iloc[:5000].reset_index(drop=True)
df_discharge_text["text"].iloc[0]

' \nName:  ___                     Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nNo Known Allergies / Adverse Drug Reactions\n \nAttending: ___\n \nChief Complaint:\nWorsening ABD distension and pain \n \nMajor Surgical or Invasive Procedure:\nParacentesis\n\n \nHistory of Present Illness:\n___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, \nbioplar, PTSD, presented from OSH ED with worsening abd \ndistension over past week.  \nPt reports self-discontinuing lasix and spirnolactone ___ weeks \nago, because she feels like "they don\'t do anything" and that \nshe "doesn\'t want to put more chemicals in her." She does not \nfollow Na-restricted diets. In the past week, she notes that she \nhas been having worsening abd distension and discomfort. She \ndenies ___ edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, \ndysuria. She had food poisoning a week ago from eatin

In [ ]:
# Create an empty data frame for results.

df_extract = df_discharge_text.drop("text", axis = 1)
df_extract

""
0
1
2
3
4
...
4995
4996
4997
4998


In [ ]:
def extract_sections(df_extract, column_name="text"):
    patterns = {
        "Chief Complaint": r'Chief Complaint:\s*(.*)',
        "Discharge Diagnosis": r'(?s)Discharge Diagnosis:\s*(.*?)\s*(?=Discharge Condition:|\Z)',
        "Present Illness History": r'(?s)History of Present Illness:\s*(.*?)\s*(Past Medical History:|\Z)',
        "Discharge Medications": r'(?s)Discharge Medications:\s*(.*?)\s*Discharge Disposition:',
        "Discharge Disposition": r'(?s)Discharge Disposition:\s*(.*?)\s*(?=Discharge Diagnosis:|\Z)',
        "Mental Status": r'(?s)Mental Status:\s*(.*?)\s*(?=Level of Consciousness:|\Z)',
        "Level of Consciousness": r'(?s)Level of Consciousness:\s*(.*?)\s*(?=Activity Status:|\Z)',
        "Discharge Instructions": r'(?s)Discharge Instructions:\s*(.*?)\s*(?=\Z)'
    }

    df_target = pd.DataFrame(index=df_extract.index)

    for col, pattern in patterns.items():
        # Extract raw text first
        df_target[col] = df_extract[column_name].apply(
            lambda text: re.search(pattern, str(text)).group(1).strip()
            if re.search(pattern, str(text)) else None
        )

        # Apply cleanup logic specific to column
        if col == "All Dx":
            df_target[col] = df_target[col].str.replace(
            r'(?i)(Primary diagnosis:|Secondary diagnosis:|Primary diagnoses:|Secondary diagnoses:|PRIMARY DIAGNOSIS|SECONDARY DIAGNOSIS|PRIMARY DIAGNOSES|SECONDARY DIAGNOSES)',
            '', regex=True) \
            .str.replace(r'(\s*\n\s*|- |-|={3,}|-{3,}|_{3,}|#|\d+)', ' ', regex=True) \
            .str.replace(r'\s{2,}', ' ', regex=True) \
            .str.strip()

        elif col == "Present Illness History":
            df_target[col] = df_target[col] \
            .str.replace(r'(\s*\n\s*|_{3,}|={3,}|\+)', ' ', regex=True) \
            .str.replace(r'\d+(\.\d+)?%?', '', regex=True) \
            .str.replace(r'\s{2,}', ' ', regex=True) \
            .str.strip()

        elif col == "Discharge Medications":
            df_target[col] = df_target[col] \
            .str.replace(r'\n', ' ', regex=True) \
            .str.replace(r'\s{2,}', ' ', regex=True) \
            .str.strip()

        elif col == "Discharge Disposition":
            df_target[col] = df_target[col] \
            .str.replace(r'\n_+|\n|:', ' ', regex=True) \
            .str.replace(r'\s{2,}', ' ', regex=True) \
            .str.strip()

        elif col == "Discharge Diagnosis":
            df_target[col] = df_target[col] \
            .str.replace(r'(\s*\n\s*|-|={3,}|-{3,}|_{3,})', ' ', regex=True) \
            .str.replace(r'\s{2,}', ' ', regex=True) \
            .str.strip()

        elif col == "Discharge Instructions":
            df_target[col] = df_target[col] \
            .str.replace(r'Follow[-\s]*up Instructions.*', ' ', regex=True) \
            .str.replace(r'\n|_{3,}| - ', ' ', regex=True) \
            .str.replace(r'\s{2,}', ' ', regex=True) \
            .str.strip()
    return df_target

extracted_final = extract_sections(df_discharge_text, column_name="text")


In [ ]:
extracted_final.iloc[6]

,6
Chief Complaint,Left hip pain
Discharge Diagnosis,Left valgus impacted femoral neck fracture
Present Illness History,"REASON CONSULT: Femur fracture history present illness: female presents with above fracture status post mechanical fall. morning, pt walking , dog pulled leash. Pt fell L hip. Immediate pain. with movement. Denies Head strike, loss consciousness blood thinners. Denies numbness weakness in extremities."
Discharge Medications,"1. Acetaminophen 1000 mg PO Q6H:PRN Pain - Mild/Fever 2. Bisacodyl 10 mg PO/PR DAILY:PRN Constipation 3. Docusate Sodium 100 mg PO BID 4. Enoxaparin Sodium 40 mg SC QHS RX *enoxaparin 40 mg/0.4 mL 40 mg Subcutaneously Nightly Disp #*30 Syringe Refills:*0 5. OxyCODONE (Immediate Release) ___ mg PO Q4H:PRN Pain - Moderate RX *oxycodone 5 mg 1 tablet(s) by mouth q4 PRN Disp #*25 Tablet Refills:*0 6. Senna 8.6 mg PO BID 7. Calcium Citrate + D (calcium citrate-vitamin D3) 315-200 mg-unit oral DAILY 8. Lactaid (lactase) 3,000 unit oral DAILY:PRN 9. Multivitamins 1 TAB PO DAILY 10. Vitamin D 400 UNIT PO DAILY"
Discharge Disposition,Home With Service Facility
Mental Status,Not provided
Level of Consciousness,Not provided
Discharge Instructions,"INSTRUCTIONS AFTER ORTHOPAEDIC SURGERY: - You were in the hospital for orthopedic surgery. It is normal to feel tired or ""washed out"" after surgery, and this feeling should improve over the first few days to week. - Resume your regular activities as tolerated, but please follow your weight bearing precautions strictly at all times. ACTIVITY AND WEIGHT BEARING: - Weightbearing as tolerated left lower extremity MEDICATIONS: 1) Take Tylenol every 6 hours around the clock. This is an over the counter medication. 2) Add oxycodone as needed for increased pain. Aim to wean off this medication in 1 week or sooner. This is an example on how to wean down: Take 1 tablet every 3 hours as needed x 1 day, then 1 tablet every 4 hours as needed x 1 day, then 1 tablet every 6 hours as needed x 1 day, then 1 tablet every 8 hours as needed x 2 days, then 1 tablet every 12 hours as needed x 1 day, then 1 tablet every before bedtime as needed x 1 day. Then continue with Tylenol for pain. 3) Do not stop the Tylenol until you are off of the narcotic medication. 4) Per state regulations, we are limited in the amount of narcotics we can prescribe. If you require more, you must contact the office to set up an appointment because we cannot refill this type of pain medication over the phone. 5) Narcotic pain relievers can cause constipation, so you should drink eight 8oz glasses of water daily and continue following the bowel regimen as stated on your medication prescription list. These meds (senna, colace, miralax) are over the counter and may be obtained at any pharmacy. 6) Do not drink alcohol, drive a motor vehicle, or operate machinery while taking narcotic pain relievers. 7) Please take all medications as prescribed by your physicians at discharge. 8) Continue all home medications unless specifically instructed to stop by your surgeon. ANTICOAGULATION: - Please take Lovenox daily for 4 weeks"
bert_embedding_history,"[0.16536308825016022, -0.11422937363386154, -0.12441807240247726, 0.0003388776967767626, 0.12136199325323105, 0.1096731573343277, 0.13097666203975677, 0.09476345777511597, 0.038855768740177155, 0.12471980601549149, 0.11792512983083725, 0.19839243590831757, 0.06213977932929993, -0.010589078068733215, -0.25270405411720276, -0.14615905284881592, 0.15119290351867676, -0.17634989321231842, -0.1049150675535202, 0.023977341130375862, -0.127333864569664, -0.059892505407333374, -0.17235560715198517, -0.08401315659284592, -0.11093340069055557, -0.2883967161178589, 0.02815384976565838, 0.7369371056556702, -0.0029084880370646715, 0.5261099338531494, 0.024426158517599106, 0.12959912419319153, 0.05748353525996208, -0.17157922685146332, -0.2291010171175003, 0.18416348099708557, 0.21721993386745453, 0.3650600016117096, 0.09010232985019684, 0.12983499467372894, 0.2133110612630844

In [ ]:
# View total missing values per column
extracted_final.isnull().sum()

# View rows with any missing values
# missing_rows = extracted_final[extracted_final.isnull().any(axis=1)]
# missing_rows

,0
Chief Complaint,178
Discharge Diagnosis,209
Present Illness History,94
Discharge Medications,166
Discharge Disposition,73
Mental Status,1139
Level of Consciousness,1194
Discharge Instructions,39


In [ ]:
extracted_final = extracted_final.dropna(subset=["Present Illness History", "Discharge Instructions"]).reset_index(drop=True)
extracted_final.isnull().sum()

,0
Chief Complaint,120
Discharge Diagnosis,202
Present Illness History,0
Discharge Medications,144
Discharge Disposition,64
Mental Status,1108
Level of Consciousness,1159
Discharge Instructions,0


In [ ]:
extracted_final.fillna("Not provided", inplace=True)
# Check for missing values
extracted_final.isnull().sum()

,0
Chief Complaint,0
Discharge Diagnosis,0
Present Illness History,0
Discharge Medications,0
Discharge Disposition,0
Mental Status,0
Level of Consciousness,0
Discharge Instructions,0


Will remove a custom list of common stopwords but not punctuation as BERT works with understanding full sentences. No lowercasing and lemmatization either. However, some additional pre-processing for Present Illness History which contains complex clinical text.

- Expand common abbreviations (keep Pt/pt/PT as is, not to confuse "patient" and "Physical Therapy", allow BERT to learn the meaning from context).

In [ ]:
abbr_dict_manual = dict(sorted({
    "ALT": "alanine aminotransferase",
    "AM": "morning",
    "ASA": "aspirin",
    "AST": "aspartate aminotransferase",
    "BID": "twice a day",
    "BM": "bowel movement",
    "BNP": "brain natriuretic peptide",
    "BRBPR": "bright red blood per rectum",
    "BP": "blood pressure",
    "BUN": "blood urea nitrogen",
    "CABG": "coronary artery bypass graft",
    "CAD": "coronary artery disease",
    "CBC": "complete blood count",
    "CHF": "congestive heart failure",
    "CKD": "chronic kidney disease",
    "COPD": "chronic obstructive pulmonary disease",
    "CP": "chest pain",
    "CT": "computed tomography",
    "CTA": "computed tomography angiography",
    "CXR": "chest x-ray",
    "DM": "diabetes mellitus",
    "DOE": "dyspnea on exertion",
    "DVT": "deep vein thrombosis",
    "ECG": "electrocardiogram",
    "ED": "emergency department",
    "EF": "ejection fraction",
    "EGD": "esophagogastroduodenoscopy",
    "EKG": "electrocardiogram",
    "EMS": "emergency medical services",
    "ER": "emergency room",
    "ERCP": "endoscopic retrograde cholangiopancreatography",
    "ESRD": "end-stage renal disease",
    "GI": "gastrointestinal",
    "HA": "headache",
    "HAART": "highly active antiretroviral therapy",
    "HD": "hemodialysis",
    "HE": "hepatic encephalopathy",
    "H/H": "hemoglobin and hematocrit",
    "HCV": "hepatitis C virus",
    "HIV": "human immunodeficiency virus",
    "HLD": "hyperlipidemia",
    "HPI": "history of present illness",
    "HR": "heart rate",
    "HTN": "hypertension",
    "ICU": "intensive care unit",
    "INR": "international normalized ratio",
    "IV": "intravenous",
    "IVDU": "intravenous drug use",
    "IVF": "intravenous fluids",
    "LAD": "left anterior descending artery",
    "LOC": "loss of consciousness",
    "LLE": "left lower extremity",
    "MICU": "medical intensive care unit",
    "MI": "myocardial infarction",
    "MRI": "magnetic resonance imaging",
    "Na": "sodium",
    "NC": "nasal cannula",
    "NG": "nasogastric",
    "NS": "normal saline",
    "O2": "oxygen",
    "OSH": "outside hospital",
    "PE": "pulmonary embolism",
    "PCP": "primary care provider",
    "PNA": "pneumonia",
    "PMH": "past medical history",
    "PND": "paroxysmal nocturnal dyspnea",
    "PO": "by mouth",
    "PTSD": "post-traumatic stress disorder",
    "RA": "room air",
    "ROS": "review of systems",
    "RR": "respiratory rate",
    "RUQ": "right upper quadrant",
    "SBP": "spontaneous bacterial peritonitis",
    "SI": "suicidal ideation",
    "SOB": "shortness of breath",
    "ST": "sinus tachycardia",
    "TIA": "transient ischemic attack",
    "UA": "urinalysis",
    "US": "ultrasound",
    "UTI": "urinary tract infection",
    "VS": "vital signs",
    "WBC": "white blood cell",
    "WNL": "within normal limits",
    "ART": "antiretroviral therapy",
    "D/C": "discharge or discontinue",
    "F/C/N/V": "fever, chills, nausea, and vomiting",
    "C/B": "complicated by",
    "H/O": "history of",
    "N/V": "nausea and vomiting",
    "HIV": "human immunodeficiency virus",
    "P/W": "presents with",
    "F/C": "fever and chills",
    "YO": "years old",
    "ABX": "antibiotics",
    "AODM": "adult onset diabetes mellitus",
    "ARDS": "acute respiratory distress syndrome",
    "AVR": "aortic valve replacement",
    "BIBA": "brought in by ambulance",
    "BPH": "benign prostatic hyperplasia",
    "CNS": "central nervous system",
    "CPAP": "continuous positive airway pressure",
    "CVA": "cerebrovascular accident",
    "DNI": "do not intubate",
    "DNR": "do not resuscitate",
    "DVT": "deep vein thrombosis",
    "Dx": "diagnosis",
    "ETOH": "alcohol",
    "FTT": "failure to thrive",
    "Fx": "fracture",
    "GSW": "gunshot wound",
    "HB": "hemoglobin",
    "HT": "height",
    "I&O": "intake and output",
    "IHD": "ischemic heart disease",
    "I/O": "input and output",
    "LFTs": "liver function tests",
    "LLQ": "left lower quadrant",
    "LUQ": "left upper quadrant",
    "MVA": "motor vehicle accident",
    "NAD": "no acute distress",
    "NPO": "nothing by mouth",
    "NTD": "no trauma detected",
    "NSR": "normal sinus rhythm",
    "ORIF": "open reduction internal fixation",
    "PERRLA": "pupils equal round reactive to light and accommodation",
    "RLE": "right lower extremity",
    "RLL": "right lower lobe",
    "RLQ": "right lower quadrant",
    "RUL": "right upper lobe",
    "RUQ": "right upper quadrant",
    "S/P": "status post",
    "TID": "three times a day",
    "TPN": "total parenteral nutrition",
    "UA": "urinalysis",
    "UOP": "urine output",
    "WNL": "within normal limits",
    "W/U": "workup",
    "XR": "x-ray",
}.items()))
# abbr_dict_manual

# Compile abbreviation pattern (case-insensitive)
abbr_pattern = re.compile(
    r'\b(' + '|'.join(re.escape(k) for k in abbr_dict_manual.keys()) + r')\b',
    flags=re.IGNORECASE
)

In [ ]:
custom_stopwords_list = [
    "a", "about", "am", "an", "and",
    "are", "been",
    "but", "by", "did", "do", "does",
    "for", "from", "had", "has", "have",
    "he", "her", "here", "hers", "herself", "him", "himself", "his",
    "into", "is", "it", "its", "itself", "just", "me", "my",
    "myself", "of", "off", "on", "or",
    "our", "ours", "ourselves", "out", "s", "she",
    "so", "some", "t", "that", "the", "their", "theirs",
    "them", "themselves", "there", "these", "they", "this", "those",
    "to", "too", "under", "up", "was", "we", "were", "what", "when",
    "where", "which", "while", "who", "whom", "will", "you", "your",
    "yours", "yourself", "yourselves"
]


In [ ]:
def preprocess_PIH(text):
    text = str(text)

    # Expand abbreviations (case-insensitive)
    text = abbr_pattern.sub(
        lambda match: abbr_dict_manual.get(match.group(0).upper(), match.group(0)),
        text
    )

    # Filter custom common words
    text = ' '.join([word for word in text.split() if word.lower() not in custom_stopwords_list])

    return text

extracted_final["Present Illness History"] = extracted_final["Present Illness History"].apply(preprocess_PIH)

In [ ]:
# Checking results
extracted_final["Present Illness History"].iloc[6]

'REASON CONSULT: Femur fracture history present illness: female presents with above fracture status post mechanical fall. morning, pt walking , dog pulled leash. Pt fell L hip. Immediate pain. with movement. Denies Head strike, loss consciousness blood thinners. Denies numbness weakness in extremities.'

In [ ]:
def generate_bert_embeddings(texts, model, tokenizer, batch_size=100):
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        tokenized = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )
        with torch.no_grad():
            outputs = model(**tokenized)
        attention_mask = tokenized["attention_mask"].unsqueeze(-1)
        masked = outputs.last_hidden_state * attention_mask
        embeddings = masked.sum(dim=1) / attention_mask.sum(dim=1)
        all_embeddings.extend(embeddings.cpu().numpy().tolist())
    return all_embeddings

# Assume all missing values are already replaced with "not provided"
texts_PIH = extracted_final["Present Illness History"].tolist()
extracted_final["bert_embedding_history"] = generate_bert_embeddings(texts_PIH, model, tokenizer)

texts_DI = extracted_final["Discharge Instructions"].tolist()
extracted_final["bert_embedding_instructions"] = generate_bert_embeddings(texts_DI, model, tokenizer)

In [ ]:
# save the final file to local drive for faster access
extracted_final.to_csv('BERT_model_5k_a.csv', index=False)

from google.colab import files
files.download('BERT_model_5k_a.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>